In [26]:
from crocoddyl import *
import pinocchio as pin
import numpy as np
from crocoddyl.diagnostic import displayTrajectory

In [27]:
# LOAD ROBOT
robot = loadKinton()
robot.initViewer(loadModel=True)
robot.display(robot.q0)

robot.framesForwardKinematics(robot.q0)

rmodel = robot.model

In [28]:
# DEFINE TARGET POSITION
target_pos  = np.array([1,0,1])
target_quat = pin.Quaternion(1, 0, 0, 0)
target_quat.normalize()

# Plot goal frame
robot.viewer.gui.addXYZaxis('world/framegoal', [1., 0., 0., 1.], .015, 2)
robot.viewer.gui.applyConfiguration('world/framegoal', target_pos.tolist() + [target_quat[0], target_quat[1], target_quat[2], target_quat[3]])
robot.viewer.gui.refresh()

In [29]:
# ACTUATION MODEL
distanceRotorCOG = 0.1525
cf = 6.6e-5
cm = 1e-6
actModel = ActuationModelUAM(robot.model, distanceRotorCOG, cf, cm)

# COST MODEL
# Create a cost model per the running and terminal action model.
runningCostModel = CostModelSum(robot.model, actModel.nu)
terminalCostModel = CostModelSum(robot.model, actModel.nu)

frameName = 'base_link'
state = StatePinocchio(robot.model)
SE3ref = pin.SE3()
SE3ref.translation = target_pos.reshape(3,1)
SE3ref.rotation = target_quat.matrix()


wBasePos  = [1]
wBaseOri  = [500]
wArmPos   = [1]
wBaseVel  = [10]
wBaseRate = [10]
wArmVel   = [10] 
stateWeights   = np.array(wBasePos * 3 + wBaseOri * 3 + wArmPos * (robot.model.nv - 6) + wBaseVel * robot.model.nv)
controlWeights = np.array([0.1]*4 + [100]*6)

goalTrackingCost = CostModelFramePlacement(rmodel,
                                           frame=rmodel.getFrameId(frameName),
                                           ref=SE3ref,
                                           nu =actModel.nu)

xRegCost = CostModelState(rmodel, 
                          state, 
                          ref=state.zero(), 
                          nu=actModel.nu,
                          activation=ActivationModelWeightedQuad(stateWeights))
uRegCost = CostModelControl(rmodel, 
                            nu=robot.
                            model.nv-2,
                            activation = ActivationModelWeightedQuad(controlWeights))
uLimCost = CostModelControl(rmodel, 
                            nu=robot.
                            model.nv-2,
                            activation = ActivationModelInequality(np.array([0.1, 0.1, 0.1, 0.1, -1, -1, -1, -1, -1, -1]), 
                                                                np.array([5, 5, 5, 5, 1, 1, 1, 1, 1, 1])))

# Then let's add the running and terminal cost functions
runningCostModel.addCost(name="pos", weight=1, cost=goalTrackingCost)
runningCostModel.addCost(name="regx", weight=1e-4, cost=xRegCost)
runningCostModel.addCost(name="regu", weight=1e-6, cost=uRegCost)
# runningCostModel.addCost(name="limu", weight=1e-3, cost=uLimCost)
terminalCostModel.addCost(name="pos", weight=1e3, cost=goalTrackingCost)

# DIFFERENTIAL ACTION MODEL
runningModel = IntegratedActionModelEuler(DifferentialActionModelActuated(robot.model, actModel, runningCostModel))
terminalModel = IntegratedActionModelEuler(DifferentialActionModelActuated(robot.model, actModel, terminalCostModel))

In [30]:
# DEFINING THE SHOOTING PROBLEM & SOLVING

# Defining the time duration for running action models and the terminal one
dt = 1e-2
runningModel.timeStep = dt

# For this optimal control problem, we define 250 knots (or running action
# models) plus a terminal knot
T = 100
q0 = rmodel.referenceConfigurations["initial_pose"].copy()
q0[:3] = np.array([[0,0,1]]).T
robot.display(q0)

v0 = pin.utils.zero(rmodel.nv)
x0 = m2a(np.concatenate([q0, v0]))
rmodel.defaultState = x0.copy()

problem = ShootingProblem(x0, [runningModel] * T, terminalModel)

# Creating the DDP solver for this OC problem, defining a logger
fddp = SolverFDDP(problem)
fddp.callback = [CallbackDDPVerbose()]
fddp.callback.append(CallbackDDPLogger())

In [31]:
us0 = [
    m.differential.quasiStatic(d.differential, rmodel.defaultState)
    if isinstance(m, IntegratedActionModelEuler) else np.zeros(0)
    for m, d in zip(fddp.problem.runningModels, fddp.problem.runningDatas)]
xs0 = [problem.initialState]*len(fddp.models())

fddp.solve()

iter 	 cost 	      stop 	    grad 	  xreg 	      ureg 	 step 	 feas
   0  6.29590e+02  7.96224e-02  2.15609e+03  1.00000e-09  1.00000e-09   0.2500     0
   1  6.30852e+02  4.80967e-02  1.26791e+03  1.00000e-08  1.00000e-08   0.0020     0
   2  6.01761e+02  4.82739e-02  1.23187e+03  1.00000e-08  1.00000e-08   0.0625     0
   3  4.40427e+03  1.06561e-01  4.42949e+05  1.00000e-07  1.00000e-07   0.0020     0
   4  5.56204e+02  9.78431e-02  4.88230e+04  1.00000e-06  1.00000e-06   0.0020     0
   5  5.32187e+02  5.97649e-02  1.43019e+03  1.00000e-06  1.00000e-06   0.0312     0
   6  4.97507e+02  1.60327e-01  1.61203e+03  1.00000e-06  1.00000e-06   0.0312     0
   7  4.74778e+02  8.38901e-02  1.91458e+03  1.00000e-06  1.00000e-06   0.0156     0
   8  3.25696e+02  5.72554e-02  9.34297e+02  1.00000e-06  1.00000e-06   0.2500     0
   9  2.67737e+02  2.86730e-01  5.26083e+03  1.00000e-06  1.00000e-06   0.0156     0
iter 	 cost 	      stop 	    grad 	  xreg 	      ureg 	 step 	 feas
  10  2.37987e

KeyboardInterrupt: 

In [19]:
displayTrajectory(robot, fddp.xs, runningModel.timeStep)

In [23]:
import time 
dt = 0.01
t = np.arange(0,1,dt)
q0 = rmodel.referenceConfigurations["initial_pose"].copy()
q0[:3] = np.array([[0,0,1]]).T

q = q0
q_d = np.zeros([rmodel.nv,1])
q_dd = np.zeros([rmodel.nv,1])
print q_d

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]


In [ ]:
nle = np.zeros([2,1])
for i in range(len(t)):
    pin.computeAllTerms(rmodel, robot.data, q, q_d)
    M = robot.data.M
    Minv = np.linalg.inv(M)
    r = np.zeros([,1])
    tau = np.zeros([2,1])
    tau[0,0] = ddp.us[i]
    nle[:,0] = m2a(robot.data.nle)    
    r[:] = tau - nle
    q_dd = np.dot(Minv, r)
    q = q + q_d*dt + q_dd*dt**2
    q_d = q_d + q_dd*dt
    #pin.forwardKinematics(robot.model, robot.data, q, q_d)
    robot.display(q)
    time.sleep(dt)